In [1]:
%pylab
import fib2
import numpy as np


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
t = np.linspace(-0.05,0.05,250, dtype=np.float32)
c, alpha = 0.76949424, 0.70087952
h1, h2 = 0.70375712, 0.47215039999999997


# Get the lightcurve
f = fib2.lc(t, analytical=1, ld_law_1=7, ldc_1 = np.array([c,alpha], dtype=np.float32))
f = np.random.normal(f, 1e-3)
fe = np.array([1e-3]*f.shape[0])
plt.scatter(t,f, c='k', s=10)

In [ ]:
for i in np.arange(16)+1:
    %timeit f = fib2.lc(t, nthreads=i, analytical=1, n_annuli=1000, ld_law_1=7, ldc_1 = np.array([c,alpha], dtype=np.float32))

In [3]:
def lnlike(theta, t, f, fe, return_model=False):
    t_zero, radius_1, k, h1, h2 =  theta
    
    if (t_zero < -0.05) or (t_zero > 0.05) : return -np.inf
    if (radius_1 < 0.0) or (radius_1 > 0.5) : return -np.inf
    if (k < 0.) or (k > 0.5) : return -np.inf
    if (h1 < 0.5) or (h1 > 1) : return -np.inf
    if (h2 < 0.3) or (h2 > 0.7) : return -np.inf
    
    
    c = 1 - h1 + h2
    alpha = np.log2(c/h2)
    if (c < 0.5) or (c > 1) : return -np.inf
    if (alpha < 0.5) or (alpha > 1) : return -np.inf    
    
    model = fib2.lc(t, nthreads=4, radius_1=radius_1, k=k, t_zero = t_zero,  analytical=1, ld_law_1=7, ldc_1 = np.array([c,alpha], dtype=np.float32))
    if return_model : return model
    else:
        chi= -0.5*np.sum( (f - model)**2 / (fe**2) )
        if np.isnan(chi) : return -np.inf
        else : return chi
    
theta = [0.0, 0.2,0.2, 0.70, 0.47]

print('Initial loglike : ',   lnlike(theta, t, f, fe, return_model=False))
model =  lnlike(theta, t, f, fe, return_model=True)
plt.scatter(t,f, c='k', s=10)
plot(t,model,'r')
    

Initial loglike :  -130.68799564141352


In [4]:
import emcee, corner
from tqdm import tqdm

nsteps = 100000
ndim = len(theta)
nwalkers =  4*ndim
p0 = [np.random.normal(theta,1e-5) for i in range(nwalkers)]
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnlike, args=[t, f, fe, False])
for pos,lnp,rstate in tqdm(sampler.sample(p0, iterations=nsteps)) : pass

100000it [01:43, 969.83it/s]


In [5]:
burn_in = 50000
samples = sampler.chain[:, burn_in:, :].reshape((-1, ndim))
fig = corner.corner(samples, labels = ['t_zero', 'radius_1', 'k', 'h1', 'h2'])


In [7]:
%timeit fib2.rv(t)


25.3 µs ± 134 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
